In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.metrics import root_mean_squared_error
import lightgbm as lgb
import pickle

In [12]:
df_train = pd.read_csv('./data/clean/diamondsdl123fexytdadcc2xso37.csv',)
df_test = pd.read_csv('./data/clean/testdl123fexydatdccso3.7x2.csv',)

y = df_train['price'].values
X = df_train.drop(['Unnamed: 0','price'], axis=1)
X_test =df_test.drop('Unnamed: 0',axis=1)

In [13]:
X_test

,carat,depth,table,x,y,z,cut,color,clarity,city,x/y,td,ad,cc
0,0.79,62.7,60.0,5.82,5.89,3.67,3.0,5.0,3.0,1.054812,0.988115,0.956938,0.006279,2.37
1,1.20,61.0,57.0,6.81,6.89,4.18,5.0,1.0,5.0,1.040138,0.988389,0.934426,0.006118,6.00
2,1.57,62.2,61.0,7.38,7.32,4.57,4.0,3.0,3.0,1.056107,1.008197,0.980707,0.006359,4.71
3,0.90,63.8,54.0,6.09,6.13,3.90,3.0,5.0,3.0,1.056107,0.993475,0.846395,0.006182,2.70
4,0.50,62.9,58.0,5.05,5.09,3.19,3.0,5.0,5.0,1.054812,0.992141,0.922099,0.006098,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30,5.0,6.0,3.0,1.054812,1.005639,0.904685,0.006069,1.71
13481,0.71,62.2,55.0,5.71,5.73,3.56,5.0,2.0,4.0,1.048770,0.996510,0.884244,0.006096,2.84
13482,0.70,61.6,55.0,5.75,5.71,3.53,5.0,5.0,5.0,1.005179,1.007005,0.892857,0.006040,3.50
13483,0.70,58.8,57.0,5.85,5.89,3.45,3.0,5.0,2.0,1.040138,0.993209,0.969388,0.005889,1.40


In [3]:
df_train.columns

Index(['Unnamed: 0', 'price', 'carat', 'depth', 'table', 'x', 'y', 'z', 'cut',
       'color', 'clarity', 'city', 'x/y', 'td', 'ad', 'cc'],
      dtype='object')

In [4]:
df_test.drop('Unnamed: 0',axis=1)

,carat,depth,table,x,y,z,cut,color,clarity,city,x/y,td,ad,cc
0,0.79,62.7,60.0,5.82,5.89,3.67,3.0,5.0,3.0,1.054812,0.988115,0.956938,0.006279,2.37
1,1.20,61.0,57.0,6.81,6.89,4.18,5.0,1.0,5.0,1.040138,0.988389,0.934426,0.006118,6.00
2,1.57,62.2,61.0,7.38,7.32,4.57,4.0,3.0,3.0,1.056107,1.008197,0.980707,0.006359,4.71
3,0.90,63.8,54.0,6.09,6.13,3.90,3.0,5.0,3.0,1.056107,0.993475,0.846395,0.006182,2.70
4,0.50,62.9,58.0,5.05,5.09,3.19,3.0,5.0,5.0,1.054812,0.992141,0.922099,0.006098,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30,5.0,6.0,3.0,1.054812,1.005639,0.904685,0.006069,1.71
13481,0.71,62.2,55.0,5.71,5.73,3.56,5.0,2.0,4.0,1.048770,0.996510,0.884244,0.006096,2.84
13482,0.70,61.6,55.0,5.75,5.71,3.53,5.0,5.0,5.0,1.005179,1.007005,0.892857,0.006040,3.50
13483,0.70,58.8,57.0,5.85,5.89,3.45,3.0,5.0,2.0,1.040138,0.993209,0.969388,0.005889,1.40


In [5]:
df_train.drop(['Unnamed: 0','price'], axis=1)

,carat,depth,table,x,y,z,cut,color,clarity,city,x/y,td,ad,cc
0,1.21,62.4,58.0,6.83,6.79,4.25,4.0,1.0,4.0,1.069918,1.005891,0.929487,0.006139,4.84
1,0.32,63.0,57.0,4.35,4.38,2.75,3.0,3.0,4.0,1.056107,0.993151,0.904762,0.006107,1.28
2,0.71,65.5,55.0,5.62,5.53,3.65,1.0,4.0,5.0,1.038843,1.016275,0.839695,0.006259,3.55
3,0.41,63.8,56.0,4.68,4.72,3.00,2.0,7.0,3.0,1.056107,0.991525,0.877743,0.006187,1.23
4,1.02,60.5,59.0,6.55,6.51,3.95,5.0,4.0,3.0,1.069918,1.006144,0.975207,0.006056,3.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,1.34,62.7,57.0,7.10,7.04,4.43,5.0,4.0,5.0,1.035822,1.008523,0.909091,0.006052,6.70
40451,2.02,57.1,60.0,8.31,8.25,4.73,2.0,5.0,2.0,1.036038,1.007273,1.050788,0.006229,4.04
40452,1.01,62.7,56.0,6.37,6.42,4.01,5.0,3.0,3.0,1.056107,0.992212,0.893142,0.006159,3.03
40453,0.33,61.9,54.3,4.45,4.47,2.76,5.0,1.0,5.0,1.056107,0.995526,0.877221,0.006011,1.65


In [42]:
sc = StandardScaler()
Xs =sc.fit_transform(X,y)
X_tests= sc.transform(X_test)


In [4]:
folds = KFold(n_splits=5, shuffle=True, random_state=2300)
params = {'num_leaves':64,
          'min_child_samples':6,
          'objective':'regression',
          'learning_rate':0.01,
          'boosting_type':'gbdt',
          'metric':'rmse',
          'max_depth':6,
          'verbose': -1}

i1=0
for trn_idx, val_idx in folds.split(X, y):
    i1 += 1
    trn_df, trn_label = X.iloc[trn_idx,:], y[trn_idx]
    val_df, val_label = X.iloc[val_idx,:], y[val_idx]
    dtrn = lgb.Dataset(trn_df,label=trn_label)
    dval = lgb.Dataset(val_df,label=val_label)
    bst = lgb.train(params,dtrn,num_boost_round=1000,valid_sets=[dtrn,dval],
    )#early_stopping_rounds=200,verbose_eval=100)
    print('-------------------',i1,'---------------------')

------------------- 1 ---------------------
------------------- 2 ---------------------
------------------- 3 ---------------------
------------------- 4 ---------------------
------------------- 5 ---------------------


In [43]:
#Set the minimum error arbitrarily large
min = -99999999999999999999999 
count = 0 #Used for keeping track of the iteration number
#How many runs to perform using randomly selected hyperparameters
iterations = 100
for i in range(iterations):
        print('iteration number', count)
        count += 1 #increment count
    #try:
             #Load in data
        mdl = lgb.LGBMRegressor(
        learning_rate = np.random.uniform(0.005, 0.015),
        boosting_type = np.random.choice(['gbdt']),
        objective = 'regression',
        metric = 'rmse',
        min_child_samples= np.random.randint(4, 8),
        num_leaves = np.random.randint(45, 85),
        max_depth = np.random.randint(4, 8),
        force_row_wise = True,)
        #iterations = np.random.randint(10, 10000))
        #Train using selected parameters
        rmse=cross_val_score(mdl,Xs,y,scoring='neg_root_mean_squared_error',cv=3,n_jobs=-1).mean()
        print('rmse:', rmse)
        if rmse > min:
            min = rmse
            pp = mdl.get_params()
    #except: #in case something goes wrong
        print('failed with')
        print(mdl.get_params())
print("*" * 50)
print('Minimum is: ', min)
print('Used params', pp)

iteration number 0
rmse: -1182.7207236769384
failed with
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.013612325501007331, 'max_depth': 6, 'min_child_samples': 5, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 63, 'objective': 'regression', 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'metric': 'rmse', 'force_row_wise': True}
iteration number 1
rmse: -2380.610311927383
failed with
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.005448066077091216, 'max_depth': 7, 'min_child_samples': 4, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 48, 'objective': 'regression', 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bi

In [38]:
model.fit(X,y)

LGBMRegressor(learning_rate=0.01, max_depth=6, metric='rmse',
              min_child_samples=6, num_leaves=64, objective='regression',
              verbose=-1)

In [29]:
#Set the minimum error arbitrarily large
min = -99999999999999999999999 
count = 0 #Used for keeping track of the iteration number
#How many runs to perform using randomly selected hyperparameters
iterations = 150
for i in range(iterations):
        print('iteration number', count)
        count += 1 #increment count
    #try:
             #Load in data
        mdl = lgb.LGBMRegressor(
        learning_rate = np.random.uniform(0.01, 0.2),
        boosting_type = np.random.choice(['gbdt']),
        objective = 'regression',
        metric = 'rmse',
        sub_feature = np.random.uniform(0, 1),
        num_leaves = np.random.randint(20, 300),
        min_data = np.random.randint(10, 100),
        max_depth = np.random.randint(150, 250),
        n_estimators = np.random.randint(30, 300),
        force_row_wise = True,
        iterations = np.random.randint(10, 10000))
        #Train using selected parameters
        rmse=cross_val_score(mdl,Xs,y,scoring='neg_root_mean_squared_error',cv=3,n_jobs=-1).mean()
        print('rmse:', rmse)
        if rmse > min:
            min = rmse
            pp = mdl.get_params()
    #except: #in case something goes wrong
        print('failed with')
        print(mdl.get_params())
print("*" * 50)
print('Minimum is: ', min)
print('Used params', pp)

iteration number 0
rmse: -657.3047121286749
failed with
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.711922078184519, 'max_depth': 68, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 190, 'n_jobs': None, 'num_leaves': 197, 'objective': 'regression', 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'metric': 'rmse', 'sub_feature': 0.7856479083526141, 'min_data': 17, 'force_row_wise': True, 'iterations': 2856}
iteration number 1
rmse: -880.3326030990389
failed with
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.6008608398628874, 'max_depth': 199, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 264, 'n_jobs': None, 'num_leaves': 142, 'objective': 'regression', 'random_state': None, 

In [ ]:
gridParams_ext = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [8,16],
    'num_leaves': [6,8], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['regression'],
    'max_bin':[255], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [500],
    'colsample_bytree' : [0.65],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2],
    'metric': ['rmse'],
    'force_col_wise': [True]}
gridParams_quick = {
    'task' : ['predict'],
    'application' : ['regression'],
    'objective' : ['root_mean_squared_error'],
    'boosting_type' : ["gbdt"],
    'num_iterations' : [2500],
    'learning_rate' : [0.05],
    'num_leaves' : [15],
    'tree_learner' : ['feature'],
    'max_depth' : [10],
    'min_data_in_leaf':[7],
    'bagging_fraction' : [1],
    'bagging_freq' : [100],
    'reg_sqrt' : ['True'],
    'metric' : ['rmse'],
    'feature_fraction' : [0.6],
    'force_col_wise': [True]}
gridParams_art={
        'num_leaves':[70,80,90],
          'min_child_samples':[5,6,7],
          'objective':['regression'],
          'learning_rate':[0.06, 0.07,0.08],
          'boosting_type':['gbdt'],
          'metric':['rmse'],
          'max_depth':[6,7,8],
          #'feature_fraction':[0.8,0.9],
          'force_col_wise' : [True],
          'bagging_fraction': [0.01,0.04],
          'device_type': ['cpu']}
gridParams_xgb= {'colsample_bytree': [0.8,0.9], 'learning_rate': [0.005,0.015], 'max_depth': [6,7,8], 'n_estimators': [800,1600], 'subsample': [0.6,0.9],
    'force_col_wise': [True]}
gridParams_empty = {'objective':['regression']}

In [ ]:
mdl = lgb.LGBMRegressor()

In [ ]:
grid = GridSearchCV(mdl,gridParams_art, verbose=10, cv=3, n_jobs=-1, scoring='neg_root_mean_squared_error')
# Run the grid
grid.fit(X, y)

best_params = grid.best_params_
best_score = grid.best_score_

print("Best parameters found: ", best_params)
print("Best score found: ", best_score)


Fitting 3 folds for each of 324 candidates, totalling 972 fits
[LightGBM] [Warning] bagging_fraction is set=0.01, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01
[LightGBM] [Warning] bagging_fraction is set=0.01, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01
[LightGBM] [Info] Total Bins 2254
[LightGBM] [Info] Number of data points in the train set: 40454, number of used features: 14
[LightGBM] [Info] Start training from score 3928.436076
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [ ]:
best_model = grid.best_estimator_

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2)

In [36]:
print('Starting predicting...')
# predict
y_pred = bst.predict(np.array(X_val))
# eval
rmse_test = root_mean_squared_error(y_val, y_pred) 
print(f'The RMSE of prediction is: {rmse_test}')

Starting predicting...
The RMSE of prediction is: 429.5377188977624


In [29]:
bst.refit(X,y)

In [39]:
preds = model.predict(X_test)
output = pd.DataFrame({'id': list(range(len(preds))),
                       'price': preds})

In [40]:
output

,id,price
0,0,6254.552125
1,1,5994.779012
2,2,11898.907132
3,3,8152.857549
4,4,7468.191417
...,...,...
13480,13480,3518.597396
13481,13481,7949.519727
13482,13482,10811.438492
13483,13483,3321.975096


In [7]:
output.to_csv('./submissions/lgbdefdl123fexydatdso3x2.csv', index=False)

In [ ]:
pickle.dump(best_model, open('./models/lgdl123dlgridtdso3.7x2_511.sav', 'wb'))

In [ ]:
train_data = lgb.Dataset('./data/clean/diamondsdlfe2xso.csv', label='price',feature_name=['price', 'carat', 'depth', 'table', 'x', 'y', 'z', 'cut', 'color',
       'clarity', 'city', 'x/y', 'td', 'ad'], categorical_feature= ['cut', 'color','clarity'])

In [ ]:
train_data

In [ ]:
validation_data = train_data.create_valid('testdl123fexydatdso3x2.csv')

In [ ]:
param = {'metric': 'rmse'}
num_round = 10

In [ ]:
bst = lgb.train(param, train_data, num_round, valid_sets=[validation_data])

[LightGBM] [Info] Construct bin mappers from text data time 0.02 seconds


TypeError: Wrong type(str) for label.
It should be list, numpy 1-D array or pandas Series